In [1]:
import imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf

In [3]:
from sklearn import datasets
dataset_iris = datasets.load_iris(return_X_y=True)

In [4]:
def generate_sample_dataset_and_feature_dict(dataset_iris):
    """
        Make dataset from iris, assuming a user likes class 0 but dislikes class 1 and class 2
    """
    npd = np.zeros(len(dataset_iris[0]), dtype=[
        ('user_id', np.int32), ('item_id', np.int32), ('label', np.int32)])
    fea_dict = dict()
    for idx in range(len(dataset_iris[0])):
        if dataset_iris[1][idx] == 0:
            npd[idx] = (0, idx, 1)
        else:
            npd[idx] = (0, idx, -1)
        fea_dict[idx] = dataset_iris[0][idx]
    return npd, fea_dict

data_all, fea_dict = generate_sample_dataset_and_feature_dict(dataset_iris)

In [5]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

112 38


In [6]:
# create dataset instance from structured ndarray data
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Test')

In [7]:
# create featurizer for mapping: (user, item) -> vector
import recsys.featurizer as featurizer
featurizer = imp.reload(featurizer)
class Featurizer(featurizer.FeaturizerBase):
    """
        Custom featurizer
    """
    def __init__(self, fea_dict):
        self.fea_dict = fea_dict

    def feature_dim(self):
        """
            Dim of X
        """
        return 4
    
    def featurize(self, user_id, item_id):
        if user_id != 0:
            raise "error"
        return self.fea_dict[item_id]

featurizer_iris = Featurizer(fea_dict)

In [8]:
# create data sampler for training and testing
import recsys.samplers.ranknet_sampler as ranknet_sampler
ranknet_sampler = imp.reload(ranknet_sampler)
train_sampler = ranknet_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = ranknet_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, seed=10)

In [9]:
# create evaluation metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [10]:
# create model
import recsys.recommenders.linear_recommender as linear_recommender
linear_recommender = imp.reload(linear_recommender)
model = linear_recommender.LinearRankNetRec(feature_dim=featurizer_iris.feature_dim(),
    init_model_dir=None, save_model_dir='LinearRankNetRec', l2_reg=0.1, train=True, serve=True)

In [11]:
# create model trainer with model
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
import datetime
print(datetime.datetime.now())

2019-07-26 10:47:58.270410


In [13]:
# train
trainer.train(total_iter=100, 
                    eval_iter=1,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 1, save_iter: 5]
[iter 1] loss: 0.761074
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 2] loss: 0.758384
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 3] loss: 0.755801
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 4] loss: 0.753320
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 5] Model saved.
[iter 5] loss: 0.750937
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 6] loss: 0.748649
..(dataset: Test) evaluation
INFO:tensorflow:Restoring para

..(dataset: Test) AUC 1.0
[iter 44] loss: 0.705100
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 45] Model saved.
[iter 45] loss: 0.704627
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 46] loss: 0.704172
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 47] loss: 0.703736
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 48] loss: 0.703317
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 49] loss: 0.702914
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
Instructions for updating:
Use standard file APIs t

..(dataset: Test) AUC 1.0
[iter 87] loss: 0.695251
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 88] loss: 0.695167
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 89] loss: 0.695087
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 90] Model saved.
[iter 90] loss: 0.695010
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 91] loss: 0.694937
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 92] loss: 0.694866
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 93] loss: 0.694798
..(dataset: Test) evaluati

In [14]:
import datetime
print(datetime.datetime.now())

2019-07-26 10:48:08.810928


In [15]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: pbModel/saved_model.pb


In [16]:
# inspect internal op tensor value of training
b = train_sampler.next_batch()
print("data:", b)
print("dy_tilde:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.fusiongraph["dy_tilde"]))

data: [([4.8, 3.1, 1.6, 0.2], [6.5, 3. , 5.5, 1.8], 1)
 ([6.5, 3. , 5.5, 1.8], [4.8, 3.1, 1.6, 0.2], 0)
 ([4.8, 3.1, 1.6, 0.2], [6.5, 3. , 5.2, 2. ], 1)
 ([6.5, 3. , 5.2, 2. ], [4.8, 3.1, 1.6, 0.2], 0)
 ([4.8, 3.1, 1.6, 0.2], [5.2, 2.7, 3.9, 1.4], 1)
 ([5.2, 2.7, 3.9, 1.4], [4.8, 3.1, 1.6, 0.2], 0)
 ([4.8, 3.1, 1.6, 0.2], [7.6, 3. , 6.6, 2.1], 1)
 ([7.6, 3. , 6.6, 2.1], [4.8, 3.1, 1.6, 0.2], 0)
 ([4.8, 3.1, 1.6, 0.2], [7.1, 3. , 5.9, 2.1], 1)
 ([7.1, 3. , 5.9, 2.1], [4.8, 3.1, 1.6, 0.2], 0)
 ([4.9, 3.1, 1.5, 0.1], [6.5, 3. , 5.5, 1.8], 1)
 ([6.5, 3. , 5.5, 1.8], [4.9, 3.1, 1.5, 0.1], 0)
 ([4.9, 3.1, 1.5, 0.1], [6.5, 3. , 5.2, 2. ], 1)
 ([6.5, 3. , 5.2, 2. ], [4.9, 3.1, 1.5, 0.1], 0)
 ([4.9, 3.1, 1.5, 0.1], [5.2, 2.7, 3.9, 1.4], 1)
 ([5.2, 2.7, 3.9, 1.4], [4.9, 3.1, 1.5, 0.1], 0)
 ([4.9, 3.1, 1.5, 0.1], [7.6, 3. , 6.6, 2.1], 1)
 ([7.6, 3. , 6.6, 2.1], [4.9, 3.1, 1.5, 0.1], 0)
 ([4.9, 3.1, 1.5, 0.1], [7.1, 3. , 5.9, 2.1], 1)
 ([7.1, 3. , 5.9, 2.1], [4.9, 3.1, 1.5, 0.1], 0)]
dy_tilde: [0.

In [17]:
# inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.fusiongraph.get_global_outputs()))

data: ([1, 1, 1, 1, 1, -1, -1, -1, -1, -1], array([([5.1, 3.5, 1.4, 0.2],), ([5. , 3. , 1.6, 0.2],),
       ([5.2, 3.5, 1.5, 0.2],), ([4.6, 3.1, 1.5, 0.2],),
       ([5.4, 3.9, 1.3, 0.4],), ([5.7, 2.8, 4.5, 1.3],),
       ([5.6, 2.8, 4.9, 2. ],), ([7.7, 2.6, 6.9, 2.3],),
       ([7.9, 3.8, 6.4, 2. ],), ([6.3, 2.7, 4.9, 1.8],)],
      dtype=[('x', '<f4', (4,))]))
score: [array([[ 0.07051609],
       [ 0.03720008],
       [ 0.06388192],
       [ 0.0401655 ],
       [ 0.08553609],
       [-0.26843032],
       [-0.34435844],
       [-0.49696603],
       [-0.39951003],
       [-0.32109332]], dtype=float32)]


In [18]:
# inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print(var)
        print(shape)
        print(trained_weight)
inspect_weights("./LinearRankNetRec/model.ckpt")

fusiongraph/Weights1dTensor/bias__0
[1]
[0.]
fusiongraph/Weights1dTensor/FC__0
[4, 1]
[[ 0.02132861]
 [ 0.02729815]
 [-0.08767033]
 [-0.05532443]]
